In [ ]:
!pip install paho-mqtt numpy scipy scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 3.9 MB/s eta 0:00:00


In [ ]:
import paho.mqtt.client as mqtt

broker = "test.mosquitto.org"
topic = "detections/demo1/rf/fused"

client = mqtt.Client()
client.connect(broker, 1883, 60)

client.publish(topic, "🚨 Threat detected from Colab!")
client.disconnect()


/tmp/ipython-input-3871648054.py:6: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [ ]:
import paho.mqtt.client as mqtt
import time
import random

# MQTT Broker details
broker = "test.mosquitto.org"
topic = "detections/demo1/rf/fused"

# Example alerts
alerts = [
    "Normal activity",
    "Suspicious RF signal",
    "Potential Threat detected",
    "Jamming attempt"
]

# Create MQTT client and connect
client = mqtt.Client()
client.connect(broker, 1883, 60)

# Loop to keep publishing messages
try:
    while True:
        msg = random.choice(alerts)
        client.publish(topic, msg)
        print("Published:", msg)
        time.sleep(5)   # wait 5 seconds before sending the next message
except KeyboardInterrupt:
    client.disconnect()
    print("Stopped publishing.")


/tmp/ipython-input-3076321374.py:18: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Published: Normal activity
Published: Suspicious RF signal
Published: Normal activity
Published: Jamming attempt
Published: Jamming attempt
Published: Suspicious RF signal
Published: Jamming attempt
Published: Suspicious RF signal
Published: Jamming attempt
Stopped publishing.


In [ ]:
# ---------------------------
# Simulated detectors
# ---------------------------

def rf_anomaly_detector():
    """Simulates RF anomaly detection based on signal strength."""
    signal_strength = random.uniform(0, 100)  # random RF signal strength
    if signal_strength > 80:
        return "RF anomaly", 70  # anomaly contributes high risk
    else:
        return "RF normal", 10   # normal contributes low risk

def wave_threat_detector():
    """Simulates digital wave threat detection based on waveform pattern."""
    pattern = random.choice([0, 1, 0, 0, 1])  # 1 = threat, 0 = normal
    if pattern == 1:
        return "Wave threat", 60  # threat contributes medium risk
    else:
        return "Wave normal", 5   # normal contributes very low risk

# ---------------------------
# Fusion Engine with score
# ---------------------------

def fusion_engine(rf_status, rf_score, wave_status, wave_score):
    """Fuses RF + Wave detections into one decision with a threat score."""
    threat_score = rf_score + wave_score  # simple additive fusion

    if threat_score >= 100:
        threat_score = 100  # cap at 100

    if threat_score > 90:
        label = "🚨 Critical Threat Detected"
    elif threat_score > 60:
        label = "⚠️ High Suspicion"
    elif threat_score > 30:
        label = "⚠️ Low Suspicion"
    else:
        label = "✅ Normal activity"

    return f"{label} | Threat Score: {threat_score}"

# ---------------------------
# MQTT Publisher
# ---------------------------

client = mqtt.Client()
client.connect(broker, 1883, 60)

print("🚀 Starting RF + Wave Threat Fusion Publisher (with Threat Score)...")

try:
    while True:
        # Detector outputs
        rf_status, rf_score = rf_anomaly_detector()
        wave_status, wave_score = wave_threat_detector()

        # Fusion decision
        fused_msg = fusion_engine(rf_status, rf_score, wave_status, wave_score)

        # Publish
        client.publish(topic, fused_msg)

        # Debug output
        print(f"[RF: {rf_status} ({rf_score}) | Wave: {wave_status} ({wave_score})] --> Published: {fused_msg}")

        time.sleep(5)

except KeyboardInterrupt:
    client.disconnect()
    print("🛑 Publisher stopped.")

/tmp/ipython-input-2968040783.py:47: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


🚀 Starting RF + Wave Threat Fusion Publisher (with Threat Score)...
[RF: RF normal (10) | Wave: Wave threat (60)] --> Published: ⚠️ High Suspicion | Threat Score: 70
[RF: RF normal (10) | Wave: Wave normal (5)] --> Published: ✅ Normal activity | Threat Score: 15
[RF: RF normal (10) | Wave: Wave threat (60)] --> Published: ⚠️ High Suspicion | Threat Score: 70
[RF: RF normal (10) | Wave: Wave normal (5)] --> Published: ✅ Normal activity | Threat Score: 15
[RF: RF normal (10) | Wave: Wave normal (5)] --> Published: ✅ Normal activity | Threat Score: 15
[RF: RF anomaly (70) | Wave: Wave threat (60)] --> Published: 🚨 Critical Threat Detected | Threat Score: 100
[RF: RF anomaly (70) | Wave: Wave normal (5)] --> Published: ⚠️ High Suspicion | Threat Score: 75
[RF: RF normal (10) | Wave: Wave normal (5)] --> Published: ✅ Normal activity | Threat Score: 15
[RF: RF normal (10) | Wave: Wave normal (5)] --> Published: ✅ Normal activity | Threat Score: 15
[RF: RF anomaly (70) | Wave: Wave normal (5)